In [1]:
import pandas as pd
import numpy as np
import os 
import json
import sys

from functools import lru_cache

from botocore.exceptions import ClientError

In [2]:
# Tells the location of folders (e.g. aws folder) - creates a path
from user_path import set_up_path 

PATH = set_up_path()
sys.path.append(PATH['tools'])
sys.path.append(PATH['tools'] + '/functions')


In [3]:
# Importing custom libraries to connect to DWS and GSheet
from functions.database import get_data_from_warehouse
from utils.gspread.gsheet_data import GetSheetData

In [4]:
# Just checking if connected
query = """select COUNT(DISTINCT stuart_package_id)
    
from dim.package p
WHERE pu_city = 'London'
LIMIT 100000
"""
data = get_data_from_warehouse(query)

Connected to the DataWarehouse
Result from query obtained
Data stored in dataframe


In [5]:
data

,count
0,19905268


# Catchment areas (CAs) and Pools

In [19]:
from password_downloader import get_passwords
# Gets a bunch of passwords. Here is specifically for the uk
PASSWORDS = get_passwords('uk') 

# PASSWORDS helps to get a connection to GSheet???
sheet = GetSheetData(DICT=PASSWORDS['gsheet_client_secret'])

In [20]:
# Connecting to GSheet
sheet_name = 'Sourcing / Onboarding Catchment Areas'

# Indicates which GSheet workbook I need
tab = 'OB Revamp Mapping'
catch_areas = sheet.worksheet_data(sheet_name, tab)

In [21]:
catch_areas

,City,Sourcing / OB Catchment Areas,StuartPoolswithinCatchmentArea,,,,,,,,
0,Basildon,Basildon,BasildonCentral,Laindon,Pitsea,,,,,,
1,Birmingham,Birmingham Central,BirminghamCentral,BirminghamYardley,BirminghamHandsworth,,,,,,
2,Birmingham,Birmingham North,BirminghamErdington,BirminghamSuttonColdfield,,,,,,,
3,Birmingham,Birmingham East,BirminghamSolihull,BirminghamHallGreen,BirminghamAcocksGreen,,,,,,
4,Birmingham,Birmingham South,BirminghamSellyOak,Northfield,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
77,Sunderland,Sunderland,SunderlandWessingtonWay,Sunderland,SunderlandRyhope,SunderlandSilksworth,,,,,
78,Teesside,Teesside,TeessideThornaby,Stocktonontees,Middlesbrough,MiddlesbroughSouth,MiddlesbroughEston,StocktonNorth,StocktonOnTees,,
79,Warrington,St Helens,StHelens,,,,,,,,
80,Warrington,Warrington City,WarringtonCentral,,,,,,,,


In [22]:
# Pivot Longer the table
catch_areas1 = catch_areas.melt(id_vars=['City','Sourcing / OB Catchment Areas'])

In [24]:
# Dropping off City and Variable 
catch_areas1.drop(columns=['variable','City'], inplace=True) 

In [25]:
catch_areas1

,Sourcing / OB Catchment Areas,value
0,Basildon,BasildonCentral
1,Birmingham Central,BirminghamCentral
2,Birmingham North,BirminghamErdington
3,Birmingham East,BirminghamSolihull
4,Birmingham South,BirminghamSellyOak
...,...,...
733,Sunderland,
734,Teesside,
735,St Helens,
736,Warrington City,


In [29]:
# Replacing empty rows with na values and dropping them
catch_areas2 = catch_areas1.replace(r'^\s*$', np.nan, regex=True).dropna(subset=['value'])

In [30]:
catch_areas2

,Sourcing / OB Catchment Areas,value
0,Basildon,BasildonCentral
1,Birmingham Central,BirminghamCentral
2,Birmingham North,BirminghamErdington
3,Birmingham East,BirminghamSolihull
4,Birmingham South,BirminghamSellyOak
...,...,...
637,Newcastle,NewcastleRisingSun
695,London Outer E,Erith
700,London Border W,Twickenham
707,London Outer N,FinsburyPark


In [32]:
# Renaming columns
catch_areas2.rename(columns={'Sourcing / OB Catchment Areas':'catchment_area',
                            'value':'stuart_delivery_area'}, inplace=True)

In [33]:
catch_areas2

,catchment_area,stuart_delivery_area
0,Basildon,BasildonCentral
1,Birmingham Central,BirminghamCentral
2,Birmingham North,BirminghamErdington
3,Birmingham East,BirminghamSolihull
4,Birmingham South,BirminghamSellyOak
...,...,...
637,Newcastle,NewcastleRisingSun
695,London Outer E,Erith
700,London Border W,Twickenham
707,London Outer N,FinsburyPark


# PC Mapping

In [ ]:
query = """SELECT COUNT(DISTINCT driver_id),
       driver_time_in_delivery_area_id,
       zone_id,
       stuart_delivery_area,
       sum(busy_s),
       sum(on_duty_s)

FROM fact.driver_time_in_delivery_area dtda
WHERE zone_id = 2
GROUP BY 2,3,4
limit 10
"""
data = get_data_from_warehouse(query)

In [ ]:
data